# 20240501
# sarah fong

- Remove adaptors from nullomer MPRA library. 
- Write .noAdaptors.fa

In [53]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import os, sys
import pandas as pd

In [54]:
PATH = "/wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/"
BUILD = "hs1"
# read
LIB_FA =os.path.join(PATH, "15mer.fo.pam.scaffold.ext200.library.TWIST.fa")
if BUILD =="hs1":
    JOINT_LIB = "/wynton/home/ahituv/fongsl/MPRA/agarwal_2023/hs1/joint_library.liftOver.to.hs1.bed" # hs1 liftover coordinates 
else:
    JOINT_LIB = "/wynton/home/ahituv/fongsl/MPRA/agarwal_2023/joint_library.bed" # hs1 liftover coordinates 

# write
LIB_FA_NO_ADAPTORS = LIB_FA.strip(".fa") + ".noAdaptors.fa" # no adaptor.fa fil
LIB_BED = LIB_FA.strip(".fa") + f"{BUILD}.bed"  # hs1.bed file for all 

In [37]:
PATH = "/wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/"

# read
LIB_FA =os.path.join(PATH, "15mer.fo.pam.scaffold.ext200.library.TWIST.fa")
JOINT_LIB = "/wynton/home/ahituv/fongsl/MPRA/agarwal_2023/hs1/joint_library.liftOver.to.hs1.bed" # hs1 liftover coordinates 

# write
LIB_FA_NO_ADAPTORS = LIB_FA.strip(".fa") + ".noAdaptors.fa" # no adaptor.fa fil
LIB_BED = LIB_FA.strip(".fa") + ".bed"  # hs1.bed 

## write fa w/ remove adaptors

In [10]:
writer = open(LIB_FA_NO_ADAPTORS, "w")

with open(LIB_FA, "r") as fa:  # read fasta file
    for value in SimpleFastaParser(fa):
        seq_id, seq = value
        
        no_adaptor = seq[15:-15]  # trim adaptors 
        
        if len(no_adaptor) == 200:
            writer.write(f">{seq_id}\n{no_adaptor}\n")
        else:
            print("something wrong", seq_id)
writer.close()

#without adaptors leng

In [18]:
%%bash 
wc -l /wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/15mer.fo.pam.scaffold.ext200.library.TWIST.noAdaptors.fa

219050 /wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/15mer.fo.pam.scaffold.ext200.library.TWIST.noAdaptors.fa


 with adaptors length

In [20]:
%%bash 
wc -l /wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/15mer.fo.pam.scaffold.ext200.library.TWIST.fa

219050 /wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/15mer.fo.pam.scaffold.ext200.library.TWIST.fa


# make mapping bed 

In [13]:
seq_ids, seqs = [], []
with open(LIB_FA, "r") as fa:  # read fasta file
    for value in SimpleFastaParser(fa):
        seq_id, seq = value
        seq_ids.append(seq_id), seqs.append(seq)

In [19]:
fa_df = pd.DataFrame(seqs, seq_ids).reset_index()
fa_df.columns=["seq.id", "seq"]
fa_df["type"] = fa_df["seq.id"].apply(lambda x: x.split("|")[0])
fa_df["name"] = fa_df["seq.id"].apply(lambda x: x.split("|")[1])
fa_df["activity"] = fa_df["seq.id"].apply(lambda x: x.split("|")[-1])
fa_df.head()

,seq.id,seq,type,name,activity
0,endog|HepG2_DNasePeakNoPromoter10535|active,AGGACCGGATCAACTGGAGAAGAGGAGGGCCCAGGATCCAGCAGAG...,endog,HepG2_DNasePeakNoPromoter10535,active
1,endog|HepG2_DNasePeakNoPromoter10586|active,AGGACCGGATCAACTCTAGAACCTATTTTTAACCGAAGAACTACTT...,endog,HepG2_DNasePeakNoPromoter10586,active
2,endog|HepG2_DNasePeakNoPromoter10714|inactive,AGGACCGGATCAACTTCCCTGTGACACAAGAGTCTTCCCAGTCTTT...,endog,HepG2_DNasePeakNoPromoter10714,inactive
3,endog|HepG2_DNasePeakNoPromoter10849|inactive,AGGACCGGATCAACTCTCTTTTTCTCTCTCTCTCTCCCCCCCACAA...,endog,HepG2_DNasePeakNoPromoter10849,inactive
4,endog|HepG2_DNasePeakNoPromoter11263|active,AGGACCGGATCAACTTCTGGAATTATAATTACCACGTGTTGGGGGA...,endog,HepG2_DNasePeakNoPromoter11263,active


## open Agarwal library bed file in hg38

In [55]:
joint = pd.read_csv(JOINT_LIB, sep='\t', header=None, names=["#chr", "start", "end", "name", "strand"])

## merge 60k MPRA library w/ scaffold library

In [56]:
fa_joint = pd.merge(joint, fa_df[["name", "seq.id", "seq"]], how="right", right_on = "name", left_on="name").drop_duplicates()
fa_joint = fa_joint.loc[~fa_joint['start'].isna()] # remove controls
fa_joint[["start", "end"]] = fa_joint[["start", "end"]].astype(int)

In [57]:
fa_joint.shape

(108359, 7)

## write library bed

In [58]:
fa_joint.to_csv(LIB_BED,sep='\t', index=False)

In [59]:
joint.loc[joint['name'].str.contains("WTC11_seq22409")]

,#chr,start,end,name,strand


In [60]:
LIB_BED

'/wynton/group/ahituv/fongsl/projects/CRISPR_nullomer_MPRA/data/20230815_nullomer_MPRA_assoc/15mer.fo.pam.scaffold.ext200.library.TWISThs1.bed'